In [1]:
import os
import sys
import requests
requests.packages.urllib3.disable_warnings()
from pyspark.ml import Pipeline#, PipelineModel#, Transformer, UnaryTransformer
from pyspark.ml.feature import VectorAssembler, StringIndexer, ChiSqSelector, StandardScaler
# from pyspark.ml.feature import VectorAssembler, VectorSizeHint, StringIndexer, ChiSqSelector
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
# , BisectingKMeans
# from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, RegressionEvaluator, MulticlassClassificationEvaluator, ClusteringEvaluator
#from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder,CrossValidator
# from pyspark.ml.param.shared import HasInputCol, HasOutputCol
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.inMemoryColumnarStorage.compressed", True)
spark.conf.set("spark.sql.inMemoryColumnarStorage.batchSize",10000)
spark.conf.set("spark.sql.shuffle.partitions",100)

In [ ]:
df = spark.read.json('medium-sparkify-event-data.json')
df = df.withColumn('sessionId', col('sessionId').cast('int')) \
    .withColumn('ts', col('ts').cast('int')).dropDuplicates()
df = df.filter(df.userId != "").persist()
ac = df.columns
train_data, test_data = df.randomSplit([.9999, .0001], seed = 1234)
df.unpersist
train_data, test_data = test_data.randomSplit([.8,.2],seed=1234)
train_data = train_data.filter(train_data.level == 'paid').persist()
col_names = []
for each in train_data.columns:
    col_names.append(each + "Index")

In [3]:
indexer = StringIndexer(inputCols = train_data.columns, outputCols = col_names, handleInvalid = 'skip')
vecAssembler = VectorAssembler(inputCols=col_names, outputCol = 'features')
standardscaler = StandardScaler().setInputCol("features").setOutputCol("Scaled_features")
css = ChiSqSelector(selectorType = 'fwe', featuresCol = 'Scaled_features', outputCol = 'Aspect', labelCol = 'levelIndex', fpr = .05)
kMeans = KMeans(featuresCol = 'Aspect', predictionCol = 'prediction', k = 3, seed = 1, initMode = 'random', maxIter = 20)
evaluator = ClusteringEvaluator(predictionCol = 'prediction', featuresCol = 'Aspect')

pipeline2 = Pipeline(stages = [indexer, vecAssembler, standardscaler, css, kMeans])

pipelineModel_paid2 = pipeline2.fit(train_data)
pred_pipelineModel_paid2 = pipelineModel_paid2.transform(train_data).cache()
evaluator.evaluate(pred_pipelineModel_paid2)


Py4JJavaError: An error occurred while calling o64.evaluate.
: java.lang.AssertionError: assertion failed: Number of clusters must be greater than one.
	at scala.Predef$.assert(Predef.scala:223)
	at org.apache.spark.ml.evaluation.SquaredEuclideanSilhouette$.computeSilhouetteScore(ClusteringEvaluator.scala:459)
	at org.apache.spark.ml.evaluation.ClusteringEvaluator.evaluate(ClusteringEvaluator.scala:115)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:64)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:564)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractComm